# NRC Emotions lexicon based text and sentiment analysis between two text corpuses
This analyses the two corpuses PA and YT on the basis of association of words with eight emotions (anger, fear, anticipation, trust, surprise, sadness, joy, and disgust) and two sentiments (negative and positive). The [NRC lexicon](https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm) is used.


# Load the python packages and initialize global variables

In [1]:
import docx
import gensim
import os
import pandas as pd
import nltk
import numpy as np
import re
import spacy
import sys
from scipy.stats import mannwhitneyu
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

nlp = spacy.load('en', disable=['parser', 'ner'])
sys.getdefaultencoding()

NRC_EMOTION_LEXICON_PATH = "Input/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"

verbs = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
nouns = ['NNS', 'NNPS']  # + 'NN' + 'NNP',
adjectives = ['JJ', 'JJR', 'JJS']

OUTPUT_PATH = "Output"
PA_CORPUS_TEXT = "PACorpus.txt"
YT_CORPUS_TEXT = "YTCorpus.txt"
INPUT_PATH = "Input"
INPUT_FILE = "US3_ALL_TRANSCRIPTS.docx"
WORD_LIST = ['i', 'me', 'our', 'my', 'we', 'us', 'you', 'your', 'she', 'her', 'he', 'him', 'his', 'they','them', 'their']

# Read given input document file 
Read the given input file and return list of lines

In [2]:
def read_input_file(file_path):
    doc = docx.Document(file_path)
    total_lines = list()
    for i in doc.paragraphs:
        total_lines.append(i.text)
    return total_lines
filename = 'Input/US3_ALL_TRANSCRIPTS.docx'
lines = read_input_file(filename)

# Grouping of Corpuses
This method groups the given text into two groups (PA and YT) and removes unnecessary lines and characters. We have list of sentences read from the docx file as argument.

In [3]:
def group_to_corpuses(lines_in_input):
    yt_group = list()
    pa_group = list()
    
    pa_grouped_by_participant = dict()
    yt_grouped_by_participant = dict()

    initial_group_flag = None
    regex = re.compile('^P[0-9]+$')
    participant = 'P01'
    for line in lines_in_input:
        # remove empty line
        if not line.strip():
            continue
        # remove line starting with Joni or Jim
        if line.strip().startswith("Joni: ") or line.strip().startswith("Jim:"):
            continue
            
        # remove line starting with R: or R : since it is the interviwer part. There is no uniformity in starting character
        # so I have to use different srating characters
        if line.strip().startswith("R:") or line.strip().startswith("R: ") or line.strip().startswith("R :"):
            continue
            
        # If R: is present in a group of lines, remove the one sentence starting with R: and leave others
        if "R:" in line:
            line_list = line.splitlines()
            lines_list = line_list
            for single_line in line_list:
                if single_line.startswith("R:"):
                    lines_list.remove(single_line)
            line = ",".join(lines_list)

        # some line have P0, P1, (P+Number) so we use regex to find matching and remove those sentences
        if re.match(regex, line):
            participant = line
            continue
        # some line with date and time stamp information of inverview are removed
        if line.strip().startswith('2018-11-') or line.strip().startswith('Total experiment talk time:'):
            continue
            
        # regex removing text between brackets
        line = re.sub(r'\[[^()]*\]', '', line)

        # replace special characters given below with comma
        line = line.replace('...', ' ,')
        line = line.replace('…', ' ,')

        # this is a case of non-alphanumeric character present in sentence which does not start with P: or YT or PA
        # convert non-alphanumeric to numeric and remove sentences starting with R : or R4. or P10(that is left over 
        # due to non-alpha numeric character)
        if not line.strip().startswith("P:") and not line.strip().startswith('YT') and not line.strip().startswith(
                'PA'):
            line = "".join([i if ord(i) < 128 else ' ' for i in line])
            if line.strip().startswith("R :") or line.strip().startswith("R4.") or line.strip().startswith("P10"):
                continue

        # check if line start with YT or PA
        # if it starts with YT set group flag to YT and keep all sentences to YT group until flag is changed to PA
        # if flag is PA, keep all sentences to PA group until flag is changed to YT
        if line.strip().lower() == 'yt':
            initial_group_flag = 'yt'
            continue
        elif line.strip().lower() == 'pa':
            initial_group_flag = 'pa'
            continue
        elif line:
            if initial_group_flag == 'pa':
                line_group = pa_grouped_by_participant.get(participant, [])
                line_group.append(line)
                pa_grouped_by_participant.update({participant: line_group})
                pa_group.append(line)
            else:
                line_group = yt_grouped_by_participant.get(participant, [])
                line_group.append(line)
                yt_grouped_by_participant.update({participant: line_group})
                yt_group.append(line)
    return pa_group, yt_group, pa_grouped_by_participant, yt_grouped_by_participant
pa_group, yt_group, pa_grouped_by_participant, yt_grouped_by_participant = group_to_corpuses(lines)

# Write to each corpus file 
write the classified text to each corpus files

In [4]:
def write_to_corpus_file(data, _type=PA_CORPUS_TEXT):
    text_file_path = os.path.join(OUTPUT_PATH, _type)
    with open(text_file_path, 'w') as outfile:
        for line in data:
            outfile.write("%s\n" % line)

In [5]:
# write to pa corpus text file
write_to_corpus_file(pa_group, PA_CORPUS_TEXT)

# write to yt corpus text file
write_to_corpus_file(yt_group, YT_CORPUS_TEXT)

# Tokenize and Clean-up
We consider:
- Each text is a list of sentences
- Each sentence is a list of tokens
- Each token is a tuple of three elements: a word form (the exact word that appeared in the text), a word lemma (a generalized version of the word), and a list of associated tags

Tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether. Gensim’s ```simple_preprocess()``` is great for this. Additionally I have set ```deacc=True``` to remove the punctuations.

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield (gensim.utils.simple_preprocess(str(sentence), deacc=True))

# Lemmatization
Lemmatization convert words to its root word. This method lemmatizes the tokenized words and joins them into sentences.
For example: ‘Studying’ becomes ‘Study’, ‘Meeting becomes ‘Meet’, ‘Better’ and ‘Best’ becomes ‘Good’.

The advantage of this is, we get to reduce the total number of unique words in the dictionary. As a result, the number of columns in the document-word matrix (created by CountVectorizer in the further step) will be denser with lesser columns.

We can expect better topics to be generated in the end.

In [7]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(" ".join(
            [token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Splitting
Splitter class use english pickle splitter to split paragraph into list of sentences and tokenize each sentence into words using ```nltk.tokenize```

In [8]:
class Splitter(object):
    def __init__(self):
        self.nltk_splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.nltk_tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self, text):
        """
        input format: a paragraph of text
        output format: a list of lists of words.
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        """
        sentences = self.nltk_splitter.tokenize(text)
        tokenized_sentences = [self.nltk_tokenizer.tokenize(sent) for sent in sentences]
        return tokenized_sentences

# POS Tagging
A POS tag is a tag that indicates the part of speech for a word. POS tags provide linguistic signal on how a word is being used within the scope of a phrase, sentence, or document. 

What I mean by this is that the word “run” can be used as a verb “I run 5 miles every day” or as a noun “I went for a run”. Sometimes the POS is very very useful in cases where it distinguishes the word sense (the meaning of the word). In other cases, it is still useful in explaining the syntactic role of a word and we can often infer semantic information from this due to our knowledge of how this syntactic role is commonly used semantically.

If we donot POS tag, performance on tasks like word sense disambiguation and sentiment analysis would be radically worse (as there are many results that show that POS provides linguistic signal that is essential for strong performance on these tasks).

Parts of Speech tagging of splitted words in a sentence by ```Splitter()```.

In [9]:
class POSTagger(object):
    def __init__(self):
        pass

    def pos_tag(self, sentences):
        """
        input format: list of lists of words
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        output format: list of lists of tagged tokens. Each tagged tokens has a
        form, a lemma, and a list of tags
            e.g: [[('this', 'this', ['DT']), ('is', 'be', ['VB']), ('a', 'a', ['DT']), ('sentence', 'sentence', ['NN'])],
                    [('this', 'this', ['DT']), ('is', 'be', ['VB']), ('another', 'another', ['DT']), ('one', 'one', ['CARD'])]]
        """

        pos = [nltk.pos_tag(sentence) for sentence in sentences]
        # adapt format
        pos = [[(word, word, [postag]) for (word, postag) in sentence] for sentence in pos]
        return pos

# Dictionary Tagging of NRC emotion lexicon 
Read the NRC emotion lexicon and tag the words with emotion and lexicon. Use this tagged data to find the emotions and sentiment of each word of the POS tagged PA and YT corpus sentences.

In [10]:
class DictionaryTagger(object):
    def __init__(self, file_path):
        self.dictionary = dict()
        self.max_key_size = 0
        with open(file_path, "r", encoding="utf-8") as nrc_file:
            for line in nrc_file.readlines():
                if not line.strip():
                    continue
                line = re.sub(r'\s+', '\t', line)
                splited = line.replace("\n", "").split("\t")
                word, emotion, value = splited[0], splited[1], splited[2]

                if word in self.dictionary.keys():
                    self.dictionary[word].append((emotion, int(value)))
                else:
                    self.dictionary[word] = [(emotion, int(value))]

    def tag(self, postagged_sentences):
        return [self.tag_sentence(sentence) for sentence in postagged_sentences]

    def tag_sentence(self, sentence, tag_with_lemmas=False):
        """
        the result is only one tagging of all the possible ones.
        The resulting tagging is determined by these two priority rules:
            - longest matches have higher priority
            - search is made from left to right
        """
        tag_sentence = []
        N = len(sentence)
        if self.max_key_size == 0:
            self.max_key_size = N
        i = 0
        while i < N:
            j = min(i + self.max_key_size, N)  # avoid overflow
            tagged = False
            while j > i:
                expression_form = ' '.join([word[0] for word in sentence[i:j]]).lower()
                expression_lemma = ' '.join([word[1] for word in sentence[i:j]]).lower()
                if tag_with_lemmas:
                    literal = expression_lemma
                else:
                    literal = expression_form
                if literal in self.dictionary:
                    # self.logger.debug("found: %s" % literal)
                    is_single_token = j - i == 1
                    original_position = i
                    i = j
                    taggings = [tag for tag in self.dictionary[literal]]
                    tagged_expression = (expression_form, expression_lemma, taggings)
                    if is_single_token:  # if the tagged literal is a single token, conserve its previous taggings:
                        original_token_tagging = sentence[original_position][2]
                        tagged_expression[2].extend(original_token_tagging)
                    tag_sentence.append(tagged_expression)
                    tagged = True
                else:
                    j = j - 1
            if not tagged:
                tag_sentence.append(sentence[i])
                i += 1
        return tag_sentence

# MannWhitney-U test
Perform MannWhitney-U test to compare the emotions and sentiments between PA corpus and YT corpus

In [11]:
def mann_whitney_u_test(group_pa, group_yt):
    print("Mann Whitney-u Test:")
    output = dict()
    for key, yt_value in group_yt.items():
        pa_value = group_pa.get(key)
        try:
            mw_stat, mw_p = mannwhitneyu(pa_value, yt_value, use_continuity=False, alternative="greater")
        except ValueError:
            mw_stat = -1  # in case of ties, Mann-Whitney cannot rank, and so cannot calculate U
            mw_p = -1
        output.update({
            key: {
                'PA': sum(pa_value),
                'YT': sum(yt_value),
                'U-value': mw_stat,
                'P-value': round(mw_p, 8)
            }
        })
    return output

# Sentiment and Emotion Score Calculation
Get sentiments and emotions for each word from dictionary tagged data.

In [12]:
def get_sentiment(sentiment):
    if not isinstance(sentiment, tuple):
        return dict()
    results = dict()
    results.update({sentiment[0]: sentiment[1]})
    return results

Aggregate the sentiment score and emotions

In [13]:
def sentiment_score(dict_tagged_sentences):
    emotions = dict()
    for sentence in dict_tagged_sentences:
        for token in sentence:
            for tag in token[2]:
                value = get_sentiment(tag)
                if not value:
                    continue
                emotions.update({tag[0]: emotions.get(tag[0], 0) + value.get(tag[0], 0)})
    return emotions

Split the sentences in each corpus using the ```Splitter()``` we defined and POS tag the words using our ```POSTagger()``` before tagging these wors with sentiments and emotions using ```DictionaryTagger()```.

In [14]:
# read NRC emotion lexicon and tag these words with the given emotions and sentiments
dicttagger = DictionaryTagger(NRC_EMOTION_LEXICON_PATH)
# initialize 
postagger = POSTagger()
splitter = Splitter()

In [15]:
pa_sentiment_full = dict()
yt_sentiment_full = dict()

# iterate throught sentences in each participant to find the score for emotions/sentiments
for participant, pa_sentences in pa_grouped_by_participant.items():
    # lemmatization of tokenized sentences in each participants in pa
    pa_sentences_group = lemmatization(sent_to_words(pa_sentences))
    pa_sentences_group = ". ".join(pa_sentences_group)

    # split the paragraphs pa corpus into sentences and each sentence are tokenized to words list
    splitted_sentences = splitter.split(pa_sentences_group)

    # POS tagging of tokenized words in sentences
    pos_tagged_sentences = postagger.pos_tag(splitted_sentences)

    # associating emotions and sentiment to the words in PA corpus using the loaded word emotion-sentiment from NRC emmotion lexicon
    dict_tagged_sentences = dicttagger.tag(pos_tagged_sentences)
    
    # calculate the scores for each participant in pa
    pa_sentiment = sentiment_score(dict_tagged_sentences)

    # get sentences for that participant from yt corpus   
    yt_sentences = yt_grouped_by_participant.get(participant, [])
    
    # lemmatization of tokenized sentences for that participants in pa
    yt_sentences_group = lemmatization(sent_to_words(yt_sentences))

    yt_sentences_group = ". ".join(yt_sentences_group)
    
    # split the paragraphs yt corpus into sentences and each sentence are tokenized to words list
    splitted_sentences = splitter.split(yt_sentences_group)

    # POS tagging of tokenized words in sentences
    pos_tagged_sentences = postagger.pos_tag(splitted_sentences)

    # associating emotions and sentiment to the words in YT corpus using the loaded word emotion-sentiment from NRC emmotion lexicon
    dict_tagged_sentences = dicttagger.tag(pos_tagged_sentences)
    
    # calculate the scores for that participant in yt
    yt_sentiment = sentiment_score(dict_tagged_sentences)
    
    # iterate through each emotion/sentiment to get the final value
    for emotion, score in yt_sentiment.items():
        # get scores vector for each emotion/sentiment from each participant in yt
        yt_scores = yt_sentiment_full.get(emotion, [])
        yt_scores.append(score)
        yt_sentiment_full.update({emotion: yt_scores})
        
        # get scores vector for each emotion/sentiment from each participant in pa
        pa_scores = pa_sentiment_full.get(emotion, [])
        pa_scores.append(pa_sentiment.get(emotion, 0))
        pa_sentiment_full.update({emotion: pa_scores})

In [16]:
print("Is PA's Postive Sentiment > YT's Postive Sentiment? %s\n" % (sum(pa_sentiment_full.get('positive')) > sum(yt_sentiment_full.get('positive'))))

# calculate the MannWhitney-U test
output = mann_whitney_u_test(pa_sentiment_full, yt_sentiment_full)
pd.DataFrame.from_dict({(i): output[i] for i in output.keys()}, orient='index')

Is PA's Postive Sentiment > YT's Postive Sentiment? True

Mann Whitney-u Test:


,PA,YT,U-value,P-value
anger,57,55,452.5,0.661203
anticipation,352,460,362.5,0.951969
disgust,202,92,655.5,0.006493
fear,153,205,341.5,0.975450
joy,210,168,516.5,0.305236
negative,167,122,493.5,0.426586
positive,939,635,591.0,0.059794
sadness,235,93,681.5,0.002170
surprise,149,87,523.0,0.272430
trust,493,416,511.0,0.333628


Null Hypothesis (𝐻0): PA has more score for sentiment/emotion than YT.

Alternative hypothesis (𝐻𝑎): PA has less score for sentiment/emotion than YT.

Null hypothesis is ``accepted for low P-value``, and ``rejected for high P-values``.

Thus with low P-values for positive sentiment, we can say ``PA has more positive sentiment than YT``.

# Removing Special characters
It is done because previous tokenization and cleaning up removes all pronuns and words not needed for word analysis eg conjunction.

This method removes the special characters, articles, un-necessary words before steming and lemmatization.

In [17]:
def remove_special_characters_from_lines(lines):
    total_clean_word_list = list()
    line_wise_word_list = list()
    for sentence in lines:
        tokens = nltk.word_tokenize(sentence)
        if 'P' in tokens:
            tokens.remove('P')
        tokens = list(filter(lambda x: x, map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x), tokens)))
        # remove articles
        tokens = [token for token in tokens if token.lower() not in ['a', 'an', 'the']]
        tagged = nltk.pos_tag(tokens)

        # lemmatize and stem the words
        stemmer = nltk.stem.PorterStemmer()
        lemmatizer = WordNetLemmatizer()

        clean_word_list = list()
        for x, y in tagged:
            x = x.lower()
            if y in adjectives:
                clean_word_list.append(lemmatizer.lemmatize(x, pos='a'))
            elif y in verbs:
                clean_word_list.append(lemmatizer.lemmatize(x, pos='v'))
            elif y in nouns:
                clean_word_list.append(stemmer.stem(x))
            else:
                clean_word_list.append(x)
        total_clean_word_list.extend(clean_word_list)
        line_wise_word_list.append(clean_word_list)
    return total_clean_word_list, line_wise_word_list

tokenize, remove special characters, stem and lemmatize

In [18]:
pa_cleaned_up, _ = remove_special_characters_from_lines(pa_group)
yt_cleaned_up, _ = remove_special_characters_from_lines(yt_group)

# Lexical Diversity Analyser

In [19]:
def lexical_diversity_analyser(words_grouping):
    return len(set(words_grouping)) / len(words_grouping)

In [20]:
# calculate the lexical diversity
pa_ld = lexical_diversity_analyser(pa_cleaned_up)
yt_ld = lexical_diversity_analyser(yt_cleaned_up)

print("PA lexical diversity: %s" % pa_ld)
print("YT lexical diversity: %s" % yt_ld)

print("\nPA>YT? %s" % (pa_ld > yt_ld))

PA lexical diversity: 0.08002443494196701
YT lexical diversity: 0.08059488245226652

PA>YT? False


# Personal Pronoun Analysis
This method counts the number of personal pronouns in two separate corpuses for analysis. The pronouns are ```'i' 'me' 'our' 'my' 'we' 'us' 'you' 'your' 'she' 'her' 'he' 'him' 'his' 'they' 'them' 'their'```

In [21]:
def personal_pronoun_analysis(words_grouping):
    pronouns_to_count = WORD_LIST
    return dict((x, words_grouping.count(x)) for x in set(pronouns_to_count))

In [22]:
pa_personal_pronoun_dict = {}
yt_personal_pronoun_dict = {}

# iterate participant wise in each corpus to get personal pronoun count
for participant, pa_sentences in pa_grouped_by_participant.items():
    # remove special characters from pa sentences of a participant
    pa_cleaned_up, _ = remove_special_characters_from_lines(pa_sentences)
    
    # get personal pronoun count for that participant in pa corpus
    pa_personal_pronoun_count = personal_pronoun_analysis(pa_cleaned_up)
        
    # remove special characters from yt sentences of that participant
    yt_cleaned_up, _ = remove_special_characters_from_lines(yt_grouped_by_participant.get(participant, []))

    # get personal pronoun count for that participant in yt corpus

    yt_personal_pronoun_count = personal_pronoun_analysis(yt_cleaned_up)
    
    for pronoun, value in pa_personal_pronoun_count.items():
        # obtain count vector of each pronoun from all participants in pa corpus
        pa_pronoun_count = pa_personal_pronoun_dict.get(pronoun, [])
        pa_pronoun_count.append(value)
        pa_personal_pronoun_dict.update({pronoun: pa_pronoun_count})
        
        # obtain count vector of each pronoun from all participants in yt corpus
        yt_pronoun_count = yt_personal_pronoun_dict.get(pronoun, [])
        yt_pronoun_count.append(yt_personal_pronoun_count.get(pronoun, 0))
        yt_personal_pronoun_dict.update({pronoun: yt_pronoun_count})

In [23]:
# calculate MannWhitney-U test
output = mann_whitney_u_test(pa_personal_pronoun_dict, yt_personal_pronoun_dict)
pd.DataFrame.from_dict({(i): output[i] for i in output.keys()}, orient='index')

Mann Whitney-u Test:


,PA,YT,U-value,P-value
he,281,28,872.5,0.000000
her,6,1,528.0,0.078305
him,34,3,740.5,0.000008
his,59,6,741.0,0.000007
i,766,842,378.0,0.925636
me,63,77,397.0,0.886665
my,44,50,442.0,0.715001
our,37,80,425.5,0.800084
she,5,2,511.0,0.156640
their,14,16,489.5,0.437796


In [24]:
print("Count of SHE in PA > YT ? %s" % (sum(pa_personal_pronoun_dict.get('she')) > sum(yt_personal_pronoun_dict.get('she'))))
print("Count of HE in PA > YT ? %s" % (sum(pa_personal_pronoun_dict.get('he')) > sum(yt_personal_pronoun_dict.get('he'))))
print("Count of HER in PA > YT ? %s" % (sum(pa_personal_pronoun_dict.get('her')) > sum(yt_personal_pronoun_dict.get('her'))))
print("Count of HIS in PA > YT ? %s\n" % (sum(pa_personal_pronoun_dict.get('his')) > sum(yt_personal_pronoun_dict.get('his'))))

Count of SHE in PA > YT ? True
Count of HE in PA > YT ? True
Count of HER in PA > YT ? True
Count of HIS in PA > YT ? True



Null Hypothesis (𝐻0): PA have more counts of personal pronouns than YT.

Alternative hypothesis (𝐻𝑎): PA have less counts of personal pronouns than YT.

Null hypothesis is ``accepted for low P-value``, and ``rejected for high P-values``.

Thus with low P-values for third person singular pronouns, we can say PA refers more to third person singular (she, he, her, his) than YT.

# Get the top 10 keywords from each topic
Top 10 keywords that are representative of the topic are created by ```show_topics()```

In [25]:
# Show top n keywords for each topic
def show_topics(vectorizer, lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

# Create the Document-Word matrix
The LDA topic model algorithm requires a document word matrix as the main input.

We can create one using CountVectorizer. In the below code, I have configured the CountVectorizer to consider words that has occurred at least 10 times (min_df), remove built-in english stopwords, convert all words to lowercase, and a word can contain numbers and alphabets of at least length 3 in order to be qualified as a word.

So, to create the doc-word matrix, we need to first initialise the CountVectorizer class with the required configuration and then apply fit_transform to actually create the matrix.

Sparsicity is nothing but the percentage of non-zero datapoints in the document-word matrix, that is data_vectorized.

In [26]:
def get_vectorized_data(data):
    data_words = list(sent_to_words(data))

    # Do lemmatization keeping only Noun, Adj, Verb, Adverb
    data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # print(data_lemmatized)

    vectorizer = CountVectorizer(
        analyzer='word',
        min_df=10,  # minimum reqd occurences of a word
        stop_words='english',  # remove stop words
        lowercase=True,  # convert all words to lowercase
        token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
    )

    data_vectorized = vectorizer.fit_transform(data_lemmatized)

    # Materialize the sparse data
    data_dense = data_vectorized.todense()

    # Compute Sparsicity = Percentage of Non-Zero cells
    print("Sparsicity: ", ((data_dense > 0).sum() / data_dense.size) * 100, "%")

    return vectorizer, data_vectorized

# GridSearch to find the best LDA model
The most important tuning parameter for LDA models is n_components (number of topics). In addition, learning_decay (which controls the learning rate) is important as well.

Besides these, other possible search params could be learning_offset (downweigh early iterations. Should be > 1) and max_iter.

The best topic model and its parameters are printed.

```Warning: process can consume a lot of time and resources.```

In [27]:
def get_optimized_lda(data):
    vectorizer, data_vectorized = get_vectorized_data(data)

    # Define Search Param
    search_params = {'n_components': [10, 15, 20, 25, 30, 35, 40], 'learning_decay': [0.1, 0.3, .5, .7, .9]}

    # Init the Model
    lda = LatentDirichletAllocation()

    # Init Grid Search Class
    model = GridSearchCV(lda, param_grid=search_params, cv=3, iid=True)

    # Do the Grid Search
    model.fit(data_vectorized)

    # Best Model
    best_lda_model = model.best_estimator_
    print(best_lda_model)
    # Model Parameters
    print("Best Model's Params: ", model.best_params_)

    # Log Likelihood Score
    print("Best Log Likelihood Score: ", model.best_score_)

    # Perplexity
    print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))
    return best_lda_model, data_vectorized, vectorizer

 # Finding dominant topic in each document
 
To classify a document as belonging to a particular topic, a logical approach is to see which topic has the highest contribution to that document and assign it.

```Note: Document here refers to each sentence of interviewee```

In [28]:
def create_document_topic_matrix(best_lda_model, data_vectorized):
    # Create Document - Topic Matrix

    lda_output = best_lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

    # index names
    docnames = ["Sentence_" + str(i) for i in range(int(lda_output.size/best_lda_model.n_components))]
    
    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)

    return df_document_topics, df_document_topic, topicnames


# Review topics distribution across documents

In [29]:
def get_topic_distribution_over_documents(df_document_topic):
    df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
    df_topic_distribution.columns = ['Topic Number', 'Number of Sentences']
    return df_topic_distribution

# Visualize the LDA model with pyLDAvis
The pyLDAvis offers the best visualization to view the topics-keywords distribution.

A good topic model will have non-overlapping, fairly big sized blobs for each topic.

In [30]:
def visualize_lda_model(best_lda_model, data_vectorized, vectorizer):
    pyLDAvis.enable_notebook()
    panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
    return panel


# Get Topic’s keywords
The weights of each keyword in each topic is contained in lda_model.components_ as a 2d array. The names of the keywords itself can be obtained from vectorizer object using get_feature_names().

Let’s use this info to construct a weight matrix for all keywords in each topic.

In [31]:
def get_topic_keywords(best_lda_model, vectorizer, topicnames):
    # Topic-Keyword Matrix
    df_topic_keywords = pd.DataFrame(best_lda_model.components_)

    # Assign Column and Index
    df_topic_keywords.columns = vectorizer.get_feature_names()
    df_topic_keywords.index = topicnames
    return df_topic_keywords

# Get the top 10 keywords of each topic

In [32]:
def get_top_ten_keywords_in_topics(best_lda_model, vectorizer):
    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=10)

    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word ' + str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic ' + str(i) for i in range(df_topic_keywords.shape[0])]
    return df_topic_keywords

# Build LDA model with sklearn
Latent Dirichlet Allocation (LDA) is initialised and fit_transform() is called to build the LDA model.

```Note: This is not used here but we can use the optimized params from GridSearch Algorithm give above to initialized and fit the model.```

In [33]:
def analyser(data):
    _, data_vectorized = get_vectorized_data(data)
    # Build LDA Model
    lda_model = LatentDirichletAllocation(
        n_components=20,  # Number of topics
        max_iter=10,  # Max learning iterations
        learning_method='online',
        random_state=100,  # Random state
        batch_size=128,  # n docs in each learning iter
        evaluate_every=-1,  # compute perplexity every n iters, default: Don't
        n_jobs=-1,  # Use all available CPUs
    )
    lda_output = lda_model.fit_transform(data_vectorized)

    print(lda_output)

    # Log Likelyhood: Higher the better
    print("Log Likelihood: ", lda_model.score(data_vectorized))

    # Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
    print("Perplexity: ", lda_model.perplexity(data_vectorized))

    # See model parameters
    pprint(lda_model.get_params())

In [34]:
# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)


def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

Get the optimized LDA parameters for each corpuses

In [35]:
print("Optimal LDA Parameters for PA corpus:")
best_lda_model_pa, data_vectorized_pa, vectorizer_pa = get_optimized_lda(pa_group)

Optimal LDA Parameters for PA corpus:
Sparsicity:  2.959816003294264 %
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.9,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)
Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -6598.710095030631
Model Perplexity:  112.19077188203974


In [36]:
print("Optimal LDA Parameters for YT corpus:")
best_lda_model_yt, data_vectorized_yt, vectorizer_yt = get_optimized_lda(yt_group)

Optimal LDA Parameters for YT corpus:
Sparsicity:  3.286978508217446 %
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.1,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)
Best Model's Params:  {'learning_decay': 0.1, 'n_components': 10}
Best Log Likelihood Score:  -6073.680005536581
Model Perplexity:  104.7416431697609


In [37]:
print("Sentence-topic matrix for PA corpus:")
df_document_topics_pa, df_document_topic_pa, topicnames_pa = create_document_topic_matrix(best_lda_model_pa, data_vectorized_pa)

df_document_topics_pa

Sentence-topic matrix for PA corpus:


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Sentence_0,0.05,0.05,0.05,0.05,0.05,0.05,0.55,0.05,0.05,0.05,6
Sentence_1,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.76,0.14,8
Sentence_2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0
Sentence_3,0.03,0.03,0.03,0.03,0.37,0.37,0.03,0.03,0.03,0.03,4
Sentence_4,0.02,0.62,0.02,0.02,0.02,0.02,0.22,0.02,0.02,0.02,1
Sentence_5,0.05,0.05,0.05,0.55,0.05,0.05,0.05,0.05,0.05,0.05,3
Sentence_6,0.3,0.01,0.35,0.01,0.01,0.01,0.01,0.01,0.26,0.01,2
Sentence_7,0.01,0.01,0.01,0.01,0.01,0.16,0.01,0.01,0.01,0.73,9
Sentence_8,0.03,0.03,0.03,0.77,0.03,0.03,0.03,0.03,0.03,0.03,3
Sentence_9,0,0,0.37,0,0,0,0,0.6,0,0,7


In [38]:
print("Participant answer with maximum value to corresponding topic in PA corpus:")
topic_sentence_map_pa = dict()
topics = list(df_document_topic_pa.columns.values)
topics = topics[0: len(topics)-1]
# iterate through the topics to find the most significant participant's sentence related to the topic in pa corpus
for topic in topics:
    index = df_document_topic_pa[topic].argmax().split("_")[1]
    maximum_value = df_document_topic_pa[topic].max()
    sentence = pa_group[int(index)]
    sentence = sentence.split("P:")[1]
    topic_sentence_map_pa.update({
        topic:{
            "value": maximum_value,
            "sentence": sentence   
        }
    })
# uncomment line below to see full sentences
pd.set_option('display.max_colwidth', -1)
pd.DataFrame.from_dict({(i): topic_sentence_map_pa[i] for i in topic_sentence_map_pa.keys()}, orient='index')

Participant answer with maximum value to corresponding topic in PA corpus:


,value,sentence
Topic0,0.97,"I'm just sort of glancing over the information that's been presented to me and I'm just sort of hoping that I'm going to glean the information that I need to glean from what is just like, what I guess is a snapshot of the things that for instance Phil here is interested in and least interested in. Alright, so research, innovation, elections, news. He likes to read about research and innovation, elections, news, Iraq. Can you speak to the fact of whether or not this is like actual ,this is an actual representation of this service? If Phil is representative of an African-American male who is 22 years old from the United States. Are these the things that he’s reading about?"
Topic1,0.94,"So, that was what that second screen, which I kind of ignore it and thought it's not that useful for me in terms of which , so basically there was that second screen which lists the videos yeah, but I think it would have been better to say.. sort of , actually list for this persona , for these types is what , for this kind of videos is what , for this persona, these types of videos is what ,"
Topic2,0.96,"'Cause I'm 22, so , . And I'm not American, but like when I'm on Facebook or something, Facebook or Twitter this seems to be the topics I see a lot where people from my age group sort of comment to Laura , politics, social media, research, politics, social issues. The most views content is about police brutality in Baltimore, Monica Harris , oh, this is the very embarrassing video of her kneeling down , yeah, making a fool of herself. But this is about police brutality, social issues. There is Syria / U.N., politics, and there is about cricket."
Topic3,0.94,"Topics that he is interested in; research, innovation, Iraq and news. Yes. Less interested in social media, social unrest in Syria. Interesting. So he's like half Interested in what we do and half not interested in what we do."
Topic4,0.92,"Yeah, for sure, definitely. I think one thing that might also be interesting is what the other interests as well, outside of YouTube and stuff like that, so I could find other stuff, so I could figure out not just what they do on YouTube, but their general interest, behaviors and patterns. That would be useful."
Topic5,0.85,"Well, he can express himself. I thought that was like , I mean, obviously, he's a college grad but still it's like ,. he's like , he actually , he could string a sentence together. That's very negative but that's something I noticed."
Topic6,0.92,"They're smart, but not necessarily , educated quotes sometimes. See this quote. I don't know what he means by satisfying an emptiness. I'm satisfied and I work to accomplish things. It feels kinda bad but really ,. okay, you are 25. Don't use 'kinda'. That's just weird."
Topic7,0.97,"Okay, the most popular to download is the Bangladesh biggest brothel, Monovan East, and then the live, and in China , oh, they're really interested , they really like the one about East documentary series about Asia, Bangladesh, China. They also like Asian shorts. This is a short. Like I think less than a ten minute view. About refugees , Okay. So, they like controversial topics about prostitution in Bangladesh, rich person, China, it's probably ,. okay , so they're not really into news, they're not even really into , like their interest is politics, but the top view, the videos that have to , what is reach for men is not news. It's not breaking news, it's not politics. It's all like human interest story. That is a professional made with like a really good cinematography."
Topic8,0.90,"Male, United States, this is male, this says 18 to 24, the same age group. And I was supposed to look for?"
Topic9,0.97,"Yes, so for me that's expected, you know, because for me I looked at and ethics and like for me this is expected. Audience size, so this is interesting for me in the campaign, so I can benchmark when I want to create a campaign or create a YouTube campaign. And I can say I can reach 

In [39]:
print("Sentence-topic matrix for YT corpus:")
df_document_topics_yt, df_document_topic_yt, topicnames_yt = create_document_topic_matrix(best_lda_model_yt, data_vectorized_yt)
df_document_topics_yt

Sentence-topic matrix for YT corpus:


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Sentence_0,0.02,0.02,0.02,0.82,0.02,0.02,0.02,0.02,0.02,0.02,3
Sentence_1,0.01,0.01,0.01,0.57,0.01,0.34,0.01,0.01,0.01,0.01,3
Sentence_2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0
Sentence_3,0.01,0.01,0.01,0.94,0.01,0.01,0.01,0.01,0.01,0.01,3
Sentence_4,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0
Sentence_5,0.03,0.03,0.03,0.03,0.03,0.7,0.03,0.03,0.03,0.03,5
Sentence_6,0.03,0.77,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,1
Sentence_7,0.01,0.44,0.17,0.01,0.34,0.01,0.01,0.01,0.01,0.01,1
Sentence_8,0.02,0.02,0.02,0.02,0.63,0.21,0.02,0.02,0.02,0.02,4
Sentence_9,0.05,0.05,0.55,0.05,0.05,0.05,0.05,0.05,0.05,0.05,2


In [40]:
print("Participant answer with maximum value to corresponding topic in YT corpus:")
topic_sentence_map_yt = dict()
topics = list(df_document_topic_yt.columns.values)
topics = topics[0: len(topics)-1]
# iterate through the topics to find the most significant participant's sentence related to the topic in yt corpus
for topic in topics:
    index = df_document_topic_yt[topic].argmax().split("_")[1]
    maximum_value = df_document_topic_yt[topic].max()
    sentence = yt_group[int(index)]
    sentence = sentence.split("P:")[1]
    topic_sentence_map_yt.update({
        topic:{
            "value": maximum_value,
            "sentence": sentence   
        }
    })
pd.DataFrame.from_dict({(i): topic_sentence_map_yt[i] for i in topic_sentence_map_yt.keys()}, orient='index')

Participant answer with maximum value to corresponding topic in YT corpus:


,value,sentence
Topic0,0.91,"59 again swept away. 59 people. Come on. They're not interested in their own people, but they're interested in other people. I'm disappointed. Let me see this one. Al Jazeera Live. Al Jazeera English Live. 0 per cent. Wow."
Topic1,0.89,"We’re looking at tools, we’re looking at a lot of tools right now to also breakdown our audience, explain our audience to us better, just so we have a better understanding of our audience. And there is another tool that I had a meeting with last week, it's cold Wirewax, it's basically a video app where engagement time on a video is increased by 8 to 9 minutes, that is exceptional. So the app will have people ,you will be able to engage with what you're watching, you’ll be able to click here or click there or click on something that you're seeing in the video, and it will blow up and will show you more information. So they’re making the viewer who is watching something continue to engage with that video content. And it's called Wirewax, because a lot of companies are spending lot of money on this app, because right now our viewing is very one dimensional, right? We produce it, we push it out, people watching it, that's it, but this is going into it, clicking on it, engaging with it."
Topic2,0.91,"Again like a title for like a news story that I think they would be interested in clicking, right, to watch?"
Topic3,0.96,"Ok, let's see, if we go to demographics, let’s check the target personas which are male, ok. Ok, we need to check viewers from Jordan, let's go to geography, there’s Jordan. I'm trying to find Jordan. So it's not among other topics, countries from viewership is coming from. Let’s see the demographics from Jordan. Ok, for male between the ages 17 to 24, 18 to 24, which is here. So it's around 15%. So in the last 90 days we had around 15% of males which is from Jordan, ok."
Topic4,0.94,"We don't know. We don't know if he is , we know he is dismembered. That, we know. But we don't know where the body is. We don't know know if it was thrown in acid. So we don't know anything."
Topic5,0.90,"I have a member of our team who wants to do more research, so they are looking for a job, and so I wanted to help him, but I said I don't know, I think every place here will need Arabic."
Topic6,0.97,"Okay, so Jordanian men from age 18 to 24 are the second most active demographic compared to other age groups. Jordan is first one. Jordan , Jordanian men from age 18 to 24 more active subscribers or viewers from all age groups are more active subscribers or viewers compared to their counterparts , to the women , second. The most popular video for them was AJ E live, and the second most viewed was about Egypt's pigeon battling video, and top three was about Jordan floods. Explain why they're important."
Topic7,0.94,"I'm just trying to figure out. I just can't ,. like this doesn't tell me anything. The only thing it tells me is 73 , so it's supposed to be Watch Time, but it says 73.4%. Per cent of what exactly. I don't get it."
Topic8,0.85,"I'm just thinking about a Youtube video title ,."
Topic9,0.96,"So, I'm supposed to memorize this place? Between 18 and 24 - 16% are male. 5% are female , Oh, they're also the most active group in terms of age. So, sorry , let put it back to before. They're not the most active group. They're the second most active age demographic who watches the , who consumes content from AJ Asia channel, so more males who watch , watch the , I mean go through AJ channel than female, and 18 to 24 is second highest, but we seem to be more popular for 25 to 34."


In [41]:
print("Topics distribution over sentences in PA corpus:")
df_topic_distribution_pa = get_topic_distribution_over_documents(df_document_topic_pa)
df_topic_distribution_pa

Topics distribution over sentences in PA corpus:


,Topic Number,Number of Sentences
0,0,299
1,4,96
2,6,77
3,1,73
4,3,69
5,2,65
6,5,60
7,8,58
8,9,56
9,7,44


In [42]:
print("Topics distribution over sentences in YT corpus:")
df_topic_distribution_yt = get_topic_distribution_over_documents(df_document_topic_yt)
df_topic_distribution_yt

Topics distribution over sentences in YT corpus:


,Topic Number,Number of Sentences
0,0,265
1,4,78
2,3,72
3,7,66
4,2,63
5,8,62
6,1,59
7,5,57
8,6,45
9,9,24


In [43]:
print("Visualization of topics and most influencial words in the topics in PA corpus:")
pa_panel = visualize_lda_model(best_lda_model_pa, data_vectorized_pa, vectorizer_pa)
filename = 'Output/lda_pa.html'
pyLDAvis.save_html(pa_panel, filename)
pyLDAvis.display(pa_panel)

Visualization of topics and most influencial words in the topics in PA corpus:


In [44]:
print("Visualization of topics and most influencial words in the topics in YT corpus:")
yt_panel = visualize_lda_model(best_lda_model_yt, data_vectorized_yt, vectorizer_yt)
pyLDAvis.save_html(yt_panel, 'Output/lda_yt.html')
pyLDAvis.display(yt_panel)

Visualization of topics and most influencial words in the topics in YT corpus:


In [45]:
print("Weights of each keyword in each topic in PA corpus:")
df_topic_keywords_pa = get_topic_keywords(best_lda_model_pa, vectorizer_pa, topicnames_pa)
df_topic_keywords_pa.head()

Weights of each keyword in each topic in PA corpus:


,actually,age,american,analytic,audience,average,base,big,click,college,...,united,use,video,view,want,watch,work,write,year,youtube
Topic0,0.100016,0.100016,2.187602,0.100004,0.100004,19.913125,0.100061,0.100005,0.100013,0.100002,...,17.354937,0.100001,16.431800,21.202825,3.410187,9.791448,13.310267,0.100012,16.506943,0.100022
Topic1,7.272042,12.011093,0.100005,1.511436,0.100008,1.286834,1.695427,10.118115,12.099947,0.100001,...,0.100001,0.100003,18.693858,2.336564,0.100015,0.100025,0.100003,0.100017,0.100003,0.100477
Topic2,1.874328,5.887047,16.012348,0.100000,0.100008,0.100004,15.401815,0.100013,0.100011,0.100015,...,1.270243,0.100003,14.146211,14.934482,1.135011,5.053783,0.100038,0.100007,0.100003,0.100010
Topic3,1.446105,0.100015,0.100018,0.100008,0.279582,0.100006,0.100000,1.058188,0.100005,0.100004,...,0.100011,0.100017,0.100027,0.100016,0.100005,0.100002,0.100010,0.100011,0.100006,0.100012
Topic4,0.100008,0.100005,0.100002,12.879187,0.100003,0.100000,0.100000,0.100000,0.100000,0.100001,...,0.100013,13.100078,3.641978,0.100013,0.100010,0.100048,0.100012,2.754249,0.100008,36.184254


In [46]:
print("Weights of each keyword in each topic in YT corpus:")
df_topic_keywords_yt = get_topic_keywords(best_lda_model_yt, vectorizer_yt, topicnames_yt)
df_topic_keywords_yt.head()

Weights of each keyword in each topic in YT corpus:


,actually,age,analytic,audience,average,big,bit,channel,click,come,...,view,viewer,want,watch,way,woman,work,write,year,youtube
Topic0,7.405142,3.713679,0.100024,0.100017,0.100000,0.814373,0.100007,1.482431,0.100005,6.706838,...,0.100009,0.100018,0.100000,10.982287,0.100006,1.205803,0.100015,23.099959,0.100002,0.144132
Topic1,0.100004,3.368991,3.565080,9.678147,0.100003,1.510154,0.278669,0.100019,11.702112,3.571007,...,1.858136,0.858620,19.314356,20.045218,0.100007,0.100000,0.805421,0.100006,37.468032,0.100006
Topic2,1.761742,1.107644,0.100009,0.100008,0.100001,0.100000,0.100006,0.100004,1.247547,0.100024,...,0.100006,0.100005,0.100012,7.559704,0.100000,0.100010,0.100011,0.100012,0.100001,0.100005
Topic3,0.100002,27.756295,1.287954,14.434805,0.100009,0.100013,0.100003,0.100007,0.100007,2.038381,...,4.753814,1.702929,0.100018,7.139894,0.100002,0.100011,0.100003,0.100003,0.731940,3.784643
Topic4,10.175454,0.100013,0.100006,30.424801,0.100004,0.100017,0.100008,4.997391,0.100023,0.100033,...,0.100020,0.100001,0.100011,0.959638,0.100000,0.100007,6.525972,0.100003,0.100000,0.100019


In [47]:
print('Top 10 keywords in each topic in PA corpus:')
df_topic_keywords_pa = get_top_ten_keywords_in_topics(best_lda_model_pa, vectorizer_pa)
df_topic_keywords_pa

Top 10 keywords in each topic in PA corpus:


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,time,minute,election,phil,view,innovation,old,average,research,state
Topic 1,jordan,video,search,don,think,number,know,persona,information,click
Topic 2,social,think,medium,politic,content,interested,issue,american,base,relate
Topic 3,interested,persona,news,topic,interesting,innovation,research,social,look,important
Topic 4,look,youtube,thank,remember,think,title,guy,thing,just,use
Topic 5,right,good,watch,college,think,grad,year,person,just,tell
Topic 6,just,know,don,really,say,work,guess,think,easy,mean
Topic 7,like,read,refugee,iraq,relation,mobile,research,view,politic,innovation
Topic 8,target,group,age,write,male,maybe,look,email,profile,man
Topic 9,audience,people,size,want,reach,mean,persona,content,different,just


In [48]:
print('Top 10 keywords in each topic in YT corpus:')
df_topic_keywords_yt = get_top_ten_keywords_in_topics(best_lda_model_yt, vectorizer_yt)
df_topic_keywords_yt

Top 10 keywords in each topic in YT corpus:


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,write,thing,interested,people,video,comment,content,news,make,email
Topic 1,video,year,information,watch,want,need,segment,old,time,mean
Topic 2,remember,thank,video,think,persona,group,target,use,right,topic
Topic 3,jordan,male,look,age,let,group,gender,day,try,think
Topic 4,know,don,audience,try,really,mean,difficult,actually,think,datum
Topic 5,look,content,easy,target,sure,audience,think,people,way,say
Topic 6,view,video,age,average,come,jordan,watch,duration,group,second
Topic 7,just,time,watch,demographic,male,total,want,thing,say,minute
Topic 8,youtube,say,man,good,analytic,watch,didn,long,search,just
Topic 9,state,united,maybe,consume,male,second,channel,female,click,age
